# Python: Mit Pandas und NumPy Zeitreihendaten bearbeiten

Dieses Notebook gibt einen kurzen Überblick wie man rohe Zeitreihen-Daten in Python referenzieren und bearbeiten kann. Dafür werden die Python-Bibliotheken  NumPy und Pandas verwendet.
* [NumPy](https://numpy.org/doc/) erlaubt die einfache Handhabung von Vektoren, Matrizen, oder generell mehrdimensionalen Datenstrukturen. Zusätzlich beinhaltet sie den Datentyp ```datetime64```, der für die Darstellung von Datums- und Uhrzeitangaben verwendet wird. Datum und Uhrzeit werden als 64-Bit-Integer Objekt gespeichert.
* [Pandas](https://pandas.pydata.org/docs/) ist das go-to Python package für Datenanalyse und -manipulation.

### Einlesen der Daten

Als Beispieldaten wurden die Daten eines Lastprofils in Amriswil aus dem Jahr 2024 verwendet und manipuliert. Die Daten wurden in einer zeitlichen Auflösung von 15 Minuten aufgenommen.

In einem ersten Schritt werden die relevanten Packages importiert. NumPy ist bereits fester Bestandteil der Python-Sprache, und muss daher nicht explizit importiert werden.

Zusätzlich werden die Daten aus Amriswil als Pandas-Dataframe eingelesen und die Kopfzeilen entsprechend angepasst. Noch erkennt Pandas nicht, dass die Kolonne ```Time``` ein ```datetime64```-Objekt beinhaltet.

In [68]:
import pandas as pd

amriswil2 = pd.read_excel("./Amriswil2.xlsx", header=None)
amriswil2.rename(columns={0:"Time", 1:"Bezug", 2:"Prod", 3:"Rueck"}, inplace=True)
amriswil2 = amriswil2[2:]

amriswil2.head()

,Time,Bezug,Prod,Rueck
2,2024-01-01 00:15:00,3.7,0,0
3,2024-01-01 00:30:00,4.5,0,0
4,2024-01-01 00:45:00,7.45,0,0
5,2024-01-01 01:00:00,6.46,0,0
6,2024-01-01 01:15:00,4.4,0,0


### Umwandlung und Referenzierung der Daten
Pandas Dataframes sind tabellarische Datenstrukturen, auf die Pandas-Funktionen angewendet werden können. 
* Die Funktion ```pd.to_datetime()``` wird auf eine Kolonne angwendet, und gibt Pandas zu erkennen, dass es sich um eine Zeitangabe handelt.
* Ohne Kontext über die Zeitzone wird ein ```datetime64```-Objekt als **naiv** bezeichnet. Die Zeitzone des Zeitstempels wird durch die Operation ```tz_localize``` spezifiziert. Aufgrund der Sommer- und Winterzeit wird ein zusätzliches Argument ```ambiguous = 'infer'``` benötigt; Pandas vermutet den Zeitpunkt der Zeitumstellung und trägt die entsprechende Verschiebung der UTC ein (von UTC+1 zu UTC+2 oder umgekehrt.)

In [69]:
# Kolonne "Time" enthält Zeitstempel und wird dementsprechend eingelesen
amriswil2["Time"] = pd.to_datetime(amriswil2["Time"]).dt.tz_localize("Europe/Zurich", ambiguous = 'infer')
amriswil2.set_index("Time", inplace = True)
amriswil2.head(5)

,Bezug,Prod,Rueck
Time,,,
2024-01-01 00:15:00+01:00,3.7,0,0
2024-01-01 00:30:00+01:00,4.5,0,0
2024-01-01 00:45:00+01:00,7.45,0,0
2024-01-01 01:00:00+01:00,6.46,0,0
2024-01-01 01:15:00+01:00,4.4,0,0


### Mithilfe einer boolschen Maskierung eine Teilmenge erhalten
Wann geschieht die Umstellung von Winterzeit in die Sommerzeit? Im Jahr 2024 wurden am 31.03.2024 die Uhren um eine Stunde zurückgestellt. Eine boolsche Maskierung kann helfen, die Umstellung sichtbar zu machen.

In [70]:
# boolsche Maskierung für die Filtrierung aller Einträge am frühen Morgen des 31.03
mask = amriswil2.index.to_series().between('2024-03-31 01:00:00', '2024-03-31 04:00:00')

amriswil31032024 = amriswil2[mask]
amriswil31032024.head(10)

,Bezug,Prod,Rueck
Time,,,
2024-03-31 01:00:00+01:00,2.54,0,0
2024-03-31 01:15:00+01:00,2.19,0.104,0
2024-03-31 01:30:00+01:00,2.21,0,0
2024-03-31 01:45:00+01:00,2.14,0,0
2024-03-31 03:00:00+02:00,2.66,0,0
2024-03-31 03:15:00+02:00,4.61,0,0
2024-03-31 03:30:00+02:00,2.5,0,0
2024-03-31 03:45:00+02:00,3.25,0,0
2024-03-31 04:00:00+02:00,2.99,0,0


### Aggregierung der Daten
Die Zeitstempel lassen sich mithilfe der Pandas-Methode ```resample('<t>')``` auf einfache Art und Weise hochaggregieren. Anstelle des Platzhalters ```<t>``` kann die temporale Auflösung angegeben werden. Die gängigsten sind die Folgenden.
* ```"min"``` für Minuten
* ```"h"``` für Stunde
* ```"D"``` für Tage
Einen extensiven Überblick bietet [diese Tabelle](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases)

Die Funktion ```.sum()``` summiert die entsprechenden Werte. Nebst ```.sum()```sind folgende Funktionen auch noch möglich:
* ```.mean()```
* ```.min()```
* ```.max()```

In [71]:
# resampling von 15 Minuten intervallen auf Stundenbasis
amriswil_hour = amriswil2.resample('h').sum()
amriswil_hour.head(20)


,Bezug,Prod,Rueck
Time,,,
2024-01-01 00:00:00+01:00,15.65,0,0
2024-01-01 01:00:00+01:00,22.8,0,0
2024-01-01 02:00:00+01:00,23.0,0,0
2024-01-01 03:00:00+01:00,22.3,0,0
2024-01-01 04:00:00+01:00,21.09,0,0
2024-01-01 05:00:00+01:00,19.25,0,0
2024-01-01 06:00:00+01:00,11.14,0,0
2024-01-01 07:00:00+01:00,10.6,0,0
2024-01-01 08:00:00+01:00,18.0,0,0
